In [14]:
import math 
import random
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [15]:
# From micrograd 
class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        # _children keeps a set of values used to create the new Value object
        self._prev = set(_children)
        # _opt keeps information on which operation was used 
        self._op = _op
        self.label = label

    # Define how we wnat to print out our Value objects 
    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        # allow addition with integers (not only Value objects)
        other = other if isinstance(other, Value) else Value(other)
        
        # return a new value object with the added value. 
        out = Value(self.data + other.data, (self, other), '+')
        # Since data is a Python number, not a Value object, we can use a typical floating point addition.  
        
        def _backward(): # same as out.grad
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __radd__(self, other):
        return self + other
        

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():  # use chain rule
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad 
        out._backward = _backward

        return out
    
    def __rmul__(self, other): # other * self
        return self * other

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward
        
        return out

        
    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    
    # Used for tanh - activation function 
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward(): # derivative of tanh is 1-tanh^2
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data 
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad = out.data * out.grad
        out._backward = _backward
    
        return out
        
    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

In [16]:
import torch 

In [17]:
# Define the input tensors and set requires_grad to True to compute gradients
x1 = torch.Tensor([2.0]).double();                 x1.requires_grad = True
x2 = torch.Tensor([0.0]).double();                 x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double();                w1.requires_grad = True
w2 = torch.Tensor([1.0]).double();                 w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double();   b.requires_grad = True
# Since thees are leaf nodes, by default, PyTorch assumes they do not need gradients. 
# Therefore, we need to explicitly say all of the nodes require gradients.

# Compute the output of the neural network
n = x1 * w1 + x2 * w2 + b
o = torch.tanh(n)

# Print the output value
print(o.data.item())

# Compute the gradients
o.backward()

# Print the gradients
print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())


0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [18]:
o # It is a tensor element and it has the backward funciton!

tensor([0.7071], dtype=torch.float64, grad_fn=<TanhBackward0>)

In [19]:
o.item() # o.item() and o.data.item() will produce the same result in PyTorch

0.7071066904050358

In [40]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        # w * x + b
        # print(list(zip(self.w, x))) # check what zip does
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out


class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs 


x = [2.0, 3.0]
n = Layer(2, 3)
n(x)

[Value(data=-0.9890202514436153),
 Value(data=0.8081731398842702),
 Value(data=-0.9931377527054055)]